## Exercice 1: Afficher les cinq premières lignes du DataFrame.



In [47]:
from pyspark.sql import *
spark = SparkSession.builder.getOrCreate()
data = spark.read.csv("demographics_de.csv",header=True,inferSchema=True)
data.printSchema()
data.createOrReplaceTempView("demo")

table = spark.table("demo") 
table.show(5)

root
 |-- state: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- age_group: string (nullable = true)
 |-- population: integer (nullable = true)

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female|    60-79|   1182736|
+------------------+------+---------+----------+
only showing top 5 rows

+------------------+------+---------+----------+
|             state|gender|age_group|population|
+------------------+------+---------+----------+
|Baden-Wuerttemberg|female|    00-04|    261674|
|Baden-Wuerttemberg|female|    05-14|    490822|
|Baden-Wuerttemberg|female|    15-34|   1293488|
|Baden-Wuerttemberg|female|    35-59|   1919649|
|Baden-Wuerttemberg|female

## Exercice 2: Compter le nombre total de lignes dans le DataFrame.



In [5]:
table.count()

192

## Exercice 3: Trouver le nombre distinct d'états (states) présents dans le DataFrame.


In [11]:
number_states = table.select('state')
number_states_distinct = number_states.distinct()
number_states_distinct.count()

16

## Exercice 4: Calculer la somme de la population totale.



In [15]:
from pyspark.sql.functions import *

sum_global_people = table.select(sum(col('population')))
sum_global_people.show()

+---------------+
|sum(population)|
+---------------+
|       83019213|
+---------------+



## Exercice 5: Afficher la population maximale par groupe d'âge (age_group).



In [21]:
max_pop_per_ga = table.select('population','age_group') \
                              .groupby('age_group') \
                              .agg(sum(col('population')) \
                              .alias('population maximale')) \
                              .sort(col('population maximale').desc()) \
                              .show()

+---------+-------------------+
|age_group|population maximale|
+---------+-------------------+
|    35-59|           29137839|
|    15-34|           19213113|
|    60-79|           17988340|
|    05-14|            7364418|
|    80-99|            5389106|
|    00-04|            3926397|
+---------+-------------------+



## Exercice 6: Filtrer les lignes du DataFrame pour n'inclure que les états (states) dont la population dépasse 1 million.



In [23]:
max_pop_per_state = table.select('population','state') \
                              .groupby('state') \
                              .agg(sum(col('population')) \
                              .alias('population maximale'))
max_pop_per_state_1M = max_pop_per_state \
                        .filter(col('population maximale') >= 1000000)
max_pop_per_state_1M.show()

+--------------------+-------------------+
|               state|population maximale|
+--------------------+-------------------+
|      Sachsen-Anhalt|            2208321|
|       Niedersachsen|            7982448|
|         Brandenburg|            2511917|
|              Berlin|            3644826|
|              Bayern|           13076721|
|             Sachsen|            4077937|
|             Hamburg|            1841179|
| Nordrhein-Westfalen|           17932651|
|  Baden-Wuerttemberg|           11069533|
|              Hessen|            6265809|
|Mecklenburg-Vorpo...|            1609675|
|  Schleswig-Holstein|            2896712|
|          Thueringen|            2143145|
|     Rheinland-Pfalz|            4084844|
+--------------------+-------------------+



## Exercice 7: Calculer la moyenne de la population pour chaque groupe d'âge (age_group).



In [46]:
poulation_per_age = table.select('population','age_group') \
                              .groupby('age_group') \
                              .agg(avg(col('population')) \
                              .alias('moyenne de population')) \
                              .sort(col('age_group').asc()) \
                              .show()


+---------+---------------------+
|age_group|moyenne de population|
+---------+---------------------+
|    00-04|         122699.90625|
|    05-14|          230138.0625|
|    15-34|         600409.78125|
|    35-59|         910557.46875|
|    60-79|           562135.625|
|    80-99|          168409.5625|
+---------+---------------------+



## Exercice 8: Trier le DataFrame par ordre décroissant de population.



In [29]:
data.sort(col('population').desc()).show()

+-------------------+------+---------+----------+
|              state|gender|age_group|population|
+-------------------+------+---------+----------+
|Nordrhein-Westfalen|female|    35-59|   3147565|
|Nordrhein-Westfalen|  male|    35-59|   3127984|
|             Bayern|  male|    35-59|   2348310|
|             Bayern|female|    35-59|   2310659|
|Nordrhein-Westfalen|  male|    15-34|   2188068|
|Nordrhein-Westfalen|female|    15-34|   2047533|
|Nordrhein-Westfalen|female|    60-79|   2009976|
| Baden-Wuerttemberg|  male|    35-59|   1955828|
| Baden-Wuerttemberg|female|    35-59|   1919649|
|Nordrhein-Westfalen|  male|    60-79|   1775275|
|             Bayern|  male|    15-34|   1639500|
|             Bayern|female|    15-34|   1507659|
| Baden-Wuerttemberg|  male|    15-34|   1423367|
|             Bayern|female|    60-79|   1421273|
|      Niedersachsen|female|    35-59|   1392647|
|      Niedersachsen|  male|    35-59|   1391992|
| Baden-Wuerttemberg|female|    15-34|   1293488|


## Exercice 9: Sélectionner les lignes du DataFrame où l'âge (age_group) est "18-24" et le sexe (gender) est "F".



In [34]:
data.filter((col('age_group') == "15-34") & (col('gender') == 'female')).show()

+--------------------+------+---------+----------+
|               state|gender|age_group|population|
+--------------------+------+---------+----------+
|  Baden-Wuerttemberg|female|    15-34|   1293488|
|              Bayern|female|    15-34|   1507659|
|              Berlin|female|    15-34|    477479|
|         Brandenburg|female|    15-34|    213065|
|              Bremen|female|    15-34|     83990|
|             Hamburg|female|    15-34|    246744|
|              Hessen|female|    15-34|    715254|
|Mecklenburg-Vorpo...|female|    15-34|    146826|
|       Niedersachsen|female|    15-34|    875509|
| Nordrhein-Westfalen|female|    15-34|   2047533|
|     Rheinland-Pfalz|female|    15-34|    448341|
|            Saarland|female|    15-34|    103235|
|             Sachsen|female|    15-34|    386154|
|      Sachsen-Anhalt|female|    15-34|    193271|
|  Schleswig-Holstein|female|    15-34|    305125|
|          Thueringen|female|    15-34|    191712|
+--------------------+------+--

## Exercice 10: Ajouter une nouvelle colonne "population_percentage" qui représente la proportion de la population par rapport à la population totale.



In [44]:
add_column = table.withColumn('population_percentage', 
                              ((col('population')/sum_global_people.collect()[0][0])*100))
add_column.show()

+------------------+------+---------+----------+---------------------+
|             state|gender|age_group|population|population_percentage|
+------------------+------+---------+----------+---------------------+
|Baden-Wuerttemberg|female|    00-04|    261674|    0.315196917128087|
|Baden-Wuerttemberg|female|    05-14|    490822|   0.5912149516522158|
|Baden-Wuerttemberg|female|    15-34|   1293488|   1.5580586146968172|
|Baden-Wuerttemberg|female|    35-59|   1919649|    2.312294866008908|
|Baden-Wuerttemberg|female|    60-79|   1182736|   1.4246533510261052|
|Baden-Wuerttemberg|female|    80-99|    419471|   0.5052697861638366|
|Baden-Wuerttemberg|  male|    00-04|    274882|   0.3311064873621483|
|Baden-Wuerttemberg|  male|    05-14|    517387|   0.6232135686470552|
|Baden-Wuerttemberg|  male|    15-34|   1423367|   1.7145031235119033|
|Baden-Wuerttemberg|  male|    35-59|   1955828|    2.355873934868547|
|Baden-Wuerttemberg|  male|    60-79|   1065669|   1.2836414144277661|
|Baden